In [40]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
sa = gspread.service_account(filename="./credentials/jp-macdonnel-b7aa48547774.json")
sh = sa.open("green-lights")
sheet = sh.worksheet("2022-10-19")

In [41]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, NoAlertPresentException, UnexpectedAlertPresentException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.set_window_position(0, 0)
driver.set_window_size(1400, 768)

In [47]:
from time import sleep
import datetime
import pandas as pd
import json

class GreenLight:

  def __init__(self, sheet):
    self.sheet = sheet
    self.df = pd.DataFrame(sheet.get_all_records())
    self.filled_df = self.df[['Customer Name', 'First Name', 'Last Name', 'Job Title', 'Work Phone', 'Address Line 1', 'City', 'State', 'Zip Code']]
    self.username = None
    self.password = None
    self.logged_in = False
    self.attempts = 0
    self.input_names = {
      "customer_name": "s_1_1_86_0",
      "first_name": "s_1_1_78_0",
      "last_name": "s_1_1_77_0",
      "job_title": "s_1_1_80_0",
      "work_phone": "s_1_1_82_0",
      "address_line_1": "s_1_1_94_0",
      "city": "s_1_1_95_0",
      "state": "s_1_1_96_0",
      "zip_code": "s_1_1_97_0",
      "acct_team_owner": "s_1_1_90_0",
      "franchise_name": "s_1_1_61_0",
      "primary_opportunity": "s_1_1_71_0",
      "expected_quantity": "s_1_1_72_0"
    }

  def attempt(self, callback):
    if self.attempts >= 15:
      self.attempts = 0
      raise Exception("attempt limit exceeded")
    print(f"attempting {callback.__name__} for the {self.attempts + 1} time")
    sleep(1)
    self.attempts += 1
    return callback()

  def get_credentials(self):
    file = open("./credentials/unishipper-credentials.json")
    data = json.load(file)
    self.username = data['username']
    self.password = data['password']

  def login(self):
    if not self.username or not self.password:
      self.get_credentials()
    driver.get("http://uone.unishippers.com")
    sleep(3)
    try:
      input_username = driver.find_element(By.ID, 's_swepi_1')
      input_password = driver.find_element(By.ID, 's_swepi_2')
      input_username.send_keys(self.username)
      input_password.send_keys(self.password)
      button_login = driver.find_element(By.ID, "s_swepi_22")
      button_login.click()
      self.attempts = 0
    except NoSuchElementException:
      if "The server you are trying to access is either busy or experiencing difficulties." in driver.find_element(By.TAG_NAME, "body").text:
        sleep(5)
        self.attempt(self.login)
      else:
        self.attempt(self.login)

  def open_leads_tab(self):
    print("opening leads tab")
    try:
      applet = driver.find_element(By.CLASS_NAME, "AppletTitle")
      if applet:
        print("applet")
        tabs = driver.find_elements(By.CLASS_NAME, "ui-corner-top")
        for tab in tabs:
          if tab.text == "Leads":
            tab.click()
            self.attempts = 0
            break
    except NoSuchElementException:
      self.attempt(self.open_leads_tab)

  def start_new_lead(self):
    try:
      button = driver.find_element(By.ID, "s_1_1_110_0_Ctrl")
      button.click()
      self.attempts = 0
    except NoSuchElementException:
      if self.attempts >= 15:
        self.attempts = 0
        raise Exception("attempt limit exceeded")
      print(f"attempting for the {self.attempts + 1} time")
      sleep(1)
      self.attempts += 1
      self.start_new_lead()

  def fill(self, customer_name, first_name, last_name, job_title, work_phone, address, city, state, zip_code):
    inputs = driver.find_elements(By.TAG_NAME, "input")

    for input in inputs:
      try:
        if input.get_attribute('name') == self.input_names['customer_name']:
          input.send_keys(customer_name)
        elif input.get_attribute('name') == self.input_names['first_name']:
          input.send_keys(first_name)
        elif input.get_attribute('name') == self.input_names['last_name']:
          input.send_keys(last_name)
        elif input.get_attribute('name') == self.input_names['job_title']:
          input.send_keys(job_title)
        elif input.get_attribute('name') == self.input_names['work_phone']:
          input.send_keys(work_phone)
        elif input.get_attribute('name') == self.input_names['address_line_1']:
          input.send_keys(address)
        elif input.get_attribute('name') == self.input_names['city']:
          input.send_keys(city)
        elif input.get_attribute('name') == self.input_names['state']:
          input.send_keys(state)
        elif input.get_attribute('name') == self.input_names['zip_code']:
          input.send_keys(zip_code)
        elif input.get_attribute('name') == self.input_names['acct_team_owner']:
          input.send_keys("JP.MacDonell")
        elif input.get_attribute('name') == self.input_names['franchise_name']:
          input.send_keys('Unishippers 1578')
        elif input.get_attribute('name') == self.input_names['primary_opportunity']:
          input.send_keys('US Express')
        elif input.get_attribute('name') == self.input_names['expected_quantity']:
          input.send_keys(100)
      except TypeError as e:
        print(e)

  def accept_alert(self):
    try:
      alert = driver.switch_to.alert
      self.attempts = 0
      status = alert.text
      alert.accept()
      return status
    except NoAlertPresentException:
      return self.attempt(self.accept_alert)

  def save_form(self):
    action = ActionChains(driver)
    action.key_down(Keys.CONTROL).send_keys('S').key_up(Keys.CONTROL).perform()

  def request_green_light(self):
    driver.find_element(By.ID, 's_1_1_48_0_Ctrl').click()

  def update_sheet(self, i, status):
    today = datetime.date.today()
    self.sheet.update(f"J{i+2}", status)
    self.sheet.update(f"K{i+2}", today.strftime("%B %d, %Y"))

  def close_popup(self, text="close"):
    for el in driver.find_elements(By.CLASS_NAME, "ui-button-text"):
      if el.text == text:
        el.click()

  def full_form(self, i, customer_name, first_name, last_name, job_title, work_phone, address, city, state, zip_code, status, updated):
    try:
      sleep(1)
      if (status):
        print(f"\n\n{i}", "\nstatus already exists:", status, updated)
      else:
        print(f"\n\n{i}", "\nno status, filling in fields")
        self.fill(customer_name, first_name, last_name, job_title, work_phone, address, city, state, zip_code)
        print("saving")
        self.save_form()
        sleep(.5)
        print("reuesting green light")
        self.request_green_light()
        sleep(.5)
        print("accepting alert")
        new_status = self.accept_alert()
        self.attempts = 0
        print(new_status)
        self.update_sheet(i, new_status)
        sleep(.5)
        print("starting new lead")
        self.start_new_lead()
    except UnexpectedAlertPresentException as e:
      print(e)
      alert = driver.switch_to.alert
      self.update_sheet(i, f"UnexpectedAlertPresentException: {alert.text}. \n\nRow may have been skipped.")
      print("unexpected alert; skipping row")
      self.update_sheet(i, "Unexpected alert error; row skipped")
      self.accept_alert()
      self.attempts = 0
    except ElementClickInterceptedException as e:
      print(e)
      self.close_popup()
      sleep(.5)
      print("saving")
      self.save_form()
      sleep(.5)
      print("reuesting green light")
      self.request_green_light()
      sleep(.5)
      print("accepting alert")
      new_status = self.accept_alert()
      self.attempts = 0
      print(new_status)
      self.update_sheet(i, new_status)
      sleep(.5)
      print("starting new lead")
      self.start_new_lead()
      try:
        self.request_green_light()
      except:
        self.update_sheet(i, "Element Click Intercepted Exception. Row likely skipped.")

  def iterate(self):
    for i, customer_name, first_name, last_name, job_title, work_phone, address, city, state, zip_code, status, updated in unishippers.df.itertuples():
      self.full_form(i, customer_name, first_name, last_name, job_title, work_phone, address, city, state, zip_code, status, updated)
      
unishippers = GreenLight(sheet)

In [43]:
unishippers.login()

In [44]:
unishippers.open_leads_tab()

opening leads tab
applet


In [45]:
unishippers.start_new_lead()

In [48]:
unishippers.iterate()



0 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


1 
status already exists: Unexpected alert error; row skipped October 22, 2022


2 
status already exists: Unexpected alert error; row skipped October 22, 2022


3 
status already exists: Unexpected alert error; row skipped October 22, 2022


4 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


5 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


6 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


7 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


8 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-00151) October 22, 2022


9 
status already exists: Your request will be sent to UPS in the next batch(SBL-EXL-0015